In [1]:
import sys
import torch
import os
from os import makedirs
import numpy as np
import open3d as o3d
from random import randint
from scipy.spatial.transform import Rotation
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2

np.set_printoptions(suppress=True)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def getWorld2View2(R, t, translate=np.array([.0, .0, .0]), scale=1.0):
    Rt = np.zeros((4, 4))
    Rt[:3, :3] = R.transpose()
    Rt[:3, 3] = t
    Rt[3, 3] = 1.0

    C2W = np.linalg.inv(Rt)
    cam_center = C2W[:3, 3]
    cam_center = (cam_center + translate) * scale
    C2W[:3, 3] = cam_center
    Rt = np.linalg.inv(C2W)
    return np.float32(Rt)

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

In [3]:
# Camera Intrinsics
rgb_camera_params = {}
depth_camera_params = {}
relative_positions = {}

def readRGBDConfig(config_file):
    # Define dictionaries to hold camera parameters
    # rgb_camera_params = {}
    # depth_camera_params = {}
    # relative_positions = {}
    
    with open(config_file, 'r') as file:
        data = file.read().split('\n\n')
    
        # Read RGB camera parameters
        rgb_data = data[0].split('\n')
        for line in rgb_data[1:4]:
            key, value = line.split('=')
            if ',' in value:
                value = tuple(map(float, value.split(',')))
            else:
                value = tuple(map(int, value.split('x')))
            rgb_camera_params[key] = value

        vFOV, hFOV = rgb_data[4].split(',')
        key, value = vFOV.split('=')
        rgb_camera_params[key] = float(value.strip('°'))
        key, value = hFOV.split('=')
        rgb_camera_params[key.strip(' ')] = float(value.strip('°'))

        # Read Depth camera parameters
        depth_data = data[1].split('\n')
        for line in depth_data[1:4]:
            key, value = line.split('=')
            if ',' in value:
                value = tuple(map(float, value.split(',')))
            else:
                value = tuple(map(int, value.split('x')))
            depth_camera_params[key] = value

        vFOV, hFOV = depth_data[4].split(',')
        key, value = vFOV.split('=')
        depth_camera_params[key] = float(value.strip('°'))
        key, value = hFOV.split('=')
        depth_camera_params[key.strip(' ')] = float(value.strip('°'))

    
        # Read relative positions of camera components
        rel_pos_data = data[2].split('\n')
        for line in rel_pos_data[1:]:
            key, value = line.split(': ')
            value = tuple(map(float, value.strip('(').strip(')').split(',')))
            relative_positions[key] = value

        # return rgb_camera_params, depth_camera_params, relative_positions
    
    # Access the loaded camera parameters
    print("RGB Camera Parameters:")
    print(rgb_camera_params)
    
    print("\nDepth Camera Parameters:")
    print(depth_camera_params)
    
    print("\nRelative Positions of Camera Components:")
    print(relative_positions)

config_file = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\config\\configuration.txt"
readRGBDConfig(config_file)

RGB Camera Parameters:
{'resolution': (1920, 1080), 'cx,cy,fx,fy': (959.5, 539.5, 960.0, 960.0), 'k1,k2,k3,p1,p2': (0.0, 0.0, 0.0, 0.0, 0.0), 'vFov': 58.7155, 'hFov': 90.0}

Depth Camera Parameters:
{'resolution': (512, 424), 'cx,cy,fx,fy': (255.5, 211.5, 256.0, 256.0), 'k1,k2,k3,p1,p2': (0.0, 0.0, 0.0, 0.0, 0.0), 'vFov': 79.2579, 'hFov': 89.9999}

Relative Positions of Camera Components:
{'RGB sensor': (0.0, 0.0, 0.0), 'RGB light source': (0.0, 0.3, 0.0), 'RGB light source size': (0.4, 0.4), 'Depth sensor': (0.03, 0.0, 0.0), 'Depth light source': (0.06, 0.0, 0.0), 'Depth light source size': (0.02, 0.02)}


In [4]:
# Camera Extrinsics
directory = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\config"

config_files = os.listdir(directory)
config_files.sort()

extrinsics = {}

# config_files[:5]
# for config_index in tqdm(range(5)):
for config_index in tqdm(range(len(config_files))):
    file = config_files[config_index]
    if file.startswith("campose-rgb-"):
        frame_id = file.split('-')[2].split('.')[0]
        config_file = os.path.join(directory, file)
        # print(config_file)
        
        with open(config_file, 'r') as file:
            lines = file.readlines()

            # Extracting position
            position_str = lines[0].replace('position=', '').split('\n')[0]
            position = np.array([float(i) for i in position_str.strip('()').split(',')])

            # Extracting rotation as a quaternion
            rotation_str = lines[1].replace('rotation_as_quaternion=', '').split('\n')[0]
            rotation = np.array([float(i) for i in rotation_str.strip('()').split(',')])

            # Extracting the 4x4 pose matrix
            pose_str = lines[3:]
            pose = np.array([[float(i) for i in row.strip('(').split(')')[0].split(',')] for row in pose_str if row != ''])    

            # print('Position:', position)
            # print('Rotation:',rotation)
            # print('Pose:',pose)
            extrinsics[frame_id] = pose

100%|██████████| 601/601 [00:00<00:00, 15053.46it/s]


In [5]:
# Directory where your images are stored
directory = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\rgb"
ply_path = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\ply"

# Get the list of files in the directory
files = os.listdir(directory)

# Sort the files to process depth and color images together
files.sort()

# Initialize an empty point cloud
point_cloud = o3d.geometry.PointCloud()

start_index = 0
ply_files = os.listdir(ply_path)
if len(ply_files) > 0:
    start_index = int(ply_files[-1].split('.')[0])

frame_step = 10
# Loop through each pair of depth and color images
# for i in tqdm(range(1)):

depth_files = [file for file in files if file.startswith("gt-rgb-depth-") ]
depth_files = depth_files[:50]

print("Number of files:", len(depth_files))

Number of files: 50


In [6]:
point_cloud = o3d.geometry.PointCloud()

# Using Open3D to create point clouds
if False:
# for i in tqdm(range(start_index, len(depth_files),frame_step)):
# for i in range(start_index, len(files),frame_step):
    depth_file_name = depth_files[i]
    # print(file)
    # if file.startswith("gt-rgb-depth-"):  # Check if the file is a depth image
    depth_file = os.path.join(directory, depth_file_name)
    
    # Get the corresponding color image
    color_file = os.path.join(directory, "rgb-" + depth_file_name[-8:])  # Assuming both files have corresponding indices
    
    frame_id = depth_file_name.split('-')[3].split('.')[0]

    # if int(frame_id) not in [146]:
    #     continue

    # Read the depth and color images
    depth_image = o3d.io.read_image(depth_file)
    color_image = o3d.io.read_image(color_file)

    # Convert images to numpy arrays
    depth_array = np.asarray(depth_image)
    color_array = np.asarray(color_image)

    width = depth_array.shape[1]
    height = depth_array.shape[0]
    
    # Intrinsic parameters of the camera (you may need to adjust these values)
    intrinsic = o3d.camera.PinholeCameraIntrinsic()
    cx = rgb_camera_params['cx,cy,fx,fy'][0]
    cy = rgb_camera_params['cx,cy,fx,fy'][1]
    fx = rgb_camera_params['cx,cy,fx,fy'][2] 
    fy = rgb_camera_params['cx,cy,fx,fy'][3]
    intrinsic.set_intrinsics(width=width, height=height, cx=cx, cy=cy, fx=fx, fy=fy)

    # print("I:", intrinsic.intrinsic_matrix)
    # print("P:", extrinsics[frame_id])

    assert frame_id in extrinsics, "No pose found for frame " + frame_id

    pose = np.linalg.inv(extrinsics[frame_id]) 

    print("Frame:", frame_id)
    print("Pose:", pose)

    # Extraction of rotation matrix (3x3 sub-matrix)
    rotation_matrix = pose[:3, :3]

    # Extraction of translation vector (last column)
    translation_vector = pose[:3, 3]

    # print(pose)
    # worldtoview = getWorld2View2(rotation_matrix, translation_vector)
    # print(rotation_matrix)
    # print(translation_vector)
    # print("W2V:", worldtoview)

    # Create a point cloud from the depth and color information
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image, depth_trunc=4.0, convert_rgb_to_intensity=False)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic,extrinsic=pose)    
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))


    # Save the point cloud to a file
    ply_file_path = os.path.join(ply_path, frame_id + ".ply")    
    # o3d.io.write_point_cloud(ply_file_path, pcd)

    # Merge current point cloud with the overall point cloud
    point_cloud += pcd

    # Clear memory
    depth_image = None
    color_image = None
    depth_array = None
    color_array = None
    pcd = None


# Visualize the final point cloud
o3d.visualization.draw_geometries([point_cloud])

In [13]:
point_cloud = o3d.geometry.PointCloud()

# Using Numpy to create point clouds
for i in tqdm(range(start_index, len(depth_files),frame_step)):
# for i in range(start_index, len(files),frame_step):
# for i in tqdm(range(1)):
    depth_file_name = depth_files[i]
    depth_file = os.path.join(directory, depth_file_name)
    
    # Get the corresponding color image
    color_file = os.path.join(directory, "rgb-" + depth_file_name[-8:])  # Assuming both files have corresponding indices
    
    frame_id = depth_file_name.split('-')[3].split('.')[0]

    # Read the depth and color images
    color_image = cv2.imread(color_file)
    depth_image = cv2.imread(depth_file, cv2.IMREAD_UNCHANGED)

    # Convert images to numpy arrays
    depth_array = np.asarray(depth_image)
    color_array = np.asarray(color_image)

    width = depth_array.shape[1]
    height = depth_array.shape[0]

    # Intrinsic parameters of the camera (you may need to adjust these values)
    intrinsic = o3d.camera.PinholeCameraIntrinsic()
    cx = rgb_camera_params['cx,cy,fx,fy'][0]
    cy = rgb_camera_params['cx,cy,fx,fy'][1]
    fx = rgb_camera_params['cx,cy,fx,fy'][2] 
    fy = rgb_camera_params['cx,cy,fx,fy'][3]
    intrinsic.set_intrinsics(width=width, height=height, cx=cx, cy=cy, fx=fx, fy=fy)
    
    # print("I:", intrinsic.intrinsic_matrix)

    assert frame_id in extrinsics, "No pose found for frame " + frame_id
    pose = extrinsics[frame_id]

    # print("Frame:", frame_id)
    # print("Pose:", pose)

    # Convert depth to 3D coordinates in camera coordinates
    u, v = np.meshgrid(np.arange(width), np.arange(height))
    u = u.flatten()
    v = v.flatten()

    # Convert depth to float32 for accurate calculations
    depth_image = depth_image.astype(np.float32)
    depth_scale = 1.0 / 1000.0 # The depth scale of the Intel RealSense D435i is 0.001
    depth = depth_image.flatten() * depth_scale
    X = (u - cx) * depth / fx
    Y = (v - cy) * depth / fy
    Z = depth

    rgb_values = color_array.reshape((-1, 3)) / 255.0
    
    # Stack the 3D coordinates
    points_camera = np.vstack((X, Y, Z, np.ones_like(X)))

    if False:
        # Extract the transformed X, Y, Z coordinates
        X_local = points_camera[0, :]
        Y_local = points_camera[1, :]
        Z_local = points_camera[2, :]
        # Stack the global coordinates
        point_cloud_local = np.vstack((X_local, Y_local, Z_local)).T
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud_local)
        pcd.transform([[1, 0, 0, 0],[0, -1, 0, 0],[0, 0, -1, 0],[0, 0, 0, 1]])
        # Add color to the point cloud
        pcd.colors = o3d.utility.Vector3dVector(rgb_values)
        point_cloud += pcd

    # Transform to global coordinates
    points_global = np.dot(pose, points_camera)

    # Extract the transformed X, Y, Z coordinates
    X_global = points_global[0, :]
    Y_global = points_global[1, :]
    Z_global = points_global[2, :]
    # Stack the global coordinates
    point_cloud_global = np.vstack((X_global, Y_global, Z_global)).T

    # Create an Open3D PointCloud
    pcd = o3d.geometry.PointCloud()
    # Set the points in the PointCloud
    pcd.points = o3d.utility.Vector3dVector(point_cloud_global)

    # Flip the point cloud
    # pcd.transform([[1, 0, 0, 0],[0, -1, 0, 0],[0, 0, -1, 0],[0, 0, 0, 1]])

    # Add color to the point cloud
    pcd.colors = o3d.utility.Vector3dVector(rgb_values)
    
    # Save the point cloud to a file
    ply_file_path = os.path.join(ply_path, frame_id + ".ply")    
    o3d.io.write_point_cloud(ply_file_path, pcd)

    # Merge current point cloud with the overall point cloud
    point_cloud += pcd

    # Clear memory
    depth_image = None
    color_image = None
    depth_array = None
    color_array = None
    pcd = None


# Create an Open3D mesh representing coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])

# Visualize the final point cloud
o3d.visualization.draw_geometries([point_cloud, axes])

pcd = None
point_cloud = None
axes = None

  0%|          | 0/5 [00:00<?, ?it/s]

I: [[960.    0.  959.5]
 [  0.  960.  539.5]
 [  0.    0.    1. ]]


 20%|██        | 1/5 [00:02<00:11,  2.80s/it]

I: [[960.    0.  959.5]
 [  0.  960.  539.5]
 [  0.    0.    1. ]]


 40%|████      | 2/5 [00:05<00:08,  2.86s/it]

...

 60%|██████    | 3/5 [00:08<00:05,  2.81s/it]

I: [[960.    0.  959.5]
 [  0.  960.  539.5]
 [  0.    0.    1. ]]


 80%|████████  | 4/5 [00:11<00:02,  2.88s/it]

I: [[960.    0.  959.5]
 [  0.  960.  539.5]
 [  0.    0.    1. ]]


100%|██████████| 5/5 [00:14<00:00,  2.87s/it]


In [ ]:
point_cloud = o3d.geometry.PointCloud()

# Using Numpy to create point clouds
for i in tqdm(range(start_index, len(depth_files),frame_step)):
# for i in range(start_index, len(files),frame_step):
# for i in tqdm(range(1)):
    depth_file_name = depth_files[i]
    depth_file = os.path.join(directory, depth_file_name)
    
    # Get the corresponding color image
    color_file = os.path.join(directory, "rgb-" + depth_file_name[-8:])  # Assuming both files have corresponding indices
    
    frame_id = depth_file_name.split('-')[3].split('.')[0]

    # Read the depth and color images
    color_image = cv2.imread(color_file)
    depth_image = cv2.imread(depth_file, cv2.IMREAD_UNCHANGED)

    # Convert images to numpy arrays
    depth_array = np.asarray(depth_image)
    color_array = np.asarray(color_image)

    width = depth_array.shape[1]
    height = depth_array.shape[0]

    # Intrinsic parameters of the camera (you may need to adjust these values)
    cx = rgb_camera_params['cx,cy,fx,fy'][0]
    cy = rgb_camera_params['cx,cy,fx,fy'][1]
    fx = rgb_camera_params['cx,cy,fx,fy'][2] 
    fy = rgb_camera_params['cx,cy,fx,fy'][3]
    
    assert frame_id in extrinsics, "No pose found for frame " + frame_id
    pose = extrinsics[frame_id]

    # print("Frame:", frame_id)
    K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    K_extended = np.eye(4)
    K_extended[:3, :3] = K
    K_inv = np.linalg.inv(K)
    M = np.dot(K_extended, pose)

    pose_inv = np.linalg.inv(pose)

    # Convert depth to 3D coordinates in camera coordinates
    u, v = np.meshgrid(np.arange(width), np.arange(height))
    u = u.flatten()
    v = v.flatten()

    # Convert depth to float32 for accurate calculations
    depth_image = depth_image.astype(np.float32)
    depth_scale = 1.0 / 1000.0
    depth = depth_image.flatten() * depth_scale
    
    uv_hom =  np.vstack((u, v, np.ones_like(u)))

    # print(np.shape(uv_hom), np.shape(K_inv))

    points_camera = np.dot(K_inv, uv_hom) * depth

    rgb_values = color_array.reshape((-1, 3)) / 255.0
    

    if False:
        # Extract the transformed X, Y, Z coordinates
        X_local = points_camera[0, :]
        Y_local = points_camera[1, :]
        Z_local = points_camera[2, :]
        # Stack the global coordinates
        point_cloud_local = np.vstack((X_local, Y_local, Z_local)).T
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud_local)
        pcd.transform([[1, 0, 0, 0],[0, -1, 0, 0],[0, 0, -1, 0],[0, 0, 0, 1]])
        # Add color to the point cloud
        pcd.colors = o3d.utility.Vector3dVector(rgb_values)
        point_cloud += pcd
    
    # print(M.shape)
    # print(points_camera.shape)
    # Transform to global coordinates
    points_camera_homogeneous = np.vstack((points_camera, np.ones(points_camera.shape[1])))
    point_3d_homogeneous_global = np.dot(pose_inv, points_camera_homogeneous)
    points_global = point_3d_homogeneous_global[:3, :] / point_3d_homogeneous_global[3, :]

    # Extract the transformed X, Y, Z coordinates
    X_global = points_global[0, :]
    Y_global = points_global[1, :]
    Z_global = points_global[2, :]
    # Stack the global coordinates
    point_cloud_global = np.vstack((X_global, Y_global, Z_global)).T

    # Create an Open3D PointCloud
    pcd = o3d.geometry.PointCloud()
    # Set the points in the PointCloud
    pcd.points = o3d.utility.Vector3dVector(point_cloud_global)

    # Flip the point cloud
    pcd.transform([[1, 0, 0, 0],[0, -1, 0, 0],[0, 0, -1, 0],[0, 0, 0, 1]])

    # Add color to the point cloud
    pcd.colors = o3d.utility.Vector3dVector(rgb_values)
    
    # Merge current point cloud with the overall point cloud
    point_cloud += pcd

    # Clear memory
    depth_image = None
    color_image = None
    depth_array = None
    color_array = None
    pcd = None


# Create an Open3D mesh representing coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])

# Visualize the final point cloud
o3d.visualization.draw_geometries([point_cloud, axes])

pcd = None
point_cloud = None
axes = None

In [ ]:
rot = qvec2rotmat([1.13133e-08,0.965926,0.258819,-4.2222e-08])
rot2 = Rotation.from_quat([1.13133e-08,0.965926,0.258819,-4.2222e-08]).as_matrix()

print(rot)
print('')
print(rot2)

In [19]:
# Visualize the final point cloud
first_ply_file_path = os.path.join(ply_path, '0000.ply')
Global_ply_path = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting"
ply_file_path = os.path.join(Global_ply_path, 'Global.ply')

# 4X4 transformation matrix
transformation_matrix =  [[1, 0, 0, 0.5],
                            [0, 1, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]]


ply_files = os.listdir(ply_path)
pcd_list = []
# for file in ply_files[:1]:
#     if file.endswith(".ply"):
#         pcd_file = os.path.join(ply_path, file)
#         pcd = o3d.io.read_point_cloud(pcd_file)
#         pcd.transform(transformation_matrix)
#         pcd_list.append(pcd)

pcd_list.append(o3d.io.read_point_cloud(ply_file_path))
# pcd_list.append(o3d.io.read_point_cloud(first_ply_file_path))

# Visualize the final point cloud
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])
o3d.visualization.draw_geometries(pcd_list)


In [24]:
# Visualize the filtered point cloud
filtered_ply_path = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\ply_filtered" 

ply_files = os.listdir(filtered_ply_path)
pcd_list = []
for file in ply_files:
    if file.endswith(".ply"):
        pcd_file = os.path.join(filtered_ply_path, file)
        pcd = o3d.io.read_point_cloud(pcd_file)
        pcd.transform(transformation_matrix)
        pcd_list.append(pcd)

# Visualize the final point cloud
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])
o3d.visualization.draw_geometries(pcd_list)
